# Summary of the notation of the paper and its translation into code

# Sets
$G=(V, A)$ is the graph, where $V=\{1, 2, \dots, N\}$. Arcs are directional, where arc $(i,j)$ leads from node $i$ to node $j$ (and vice versa). All arcs can be travelled in both directions.

The journey of one train, its path, is denoted as $P=\{1, i_1, i_2, \dots, N\}$, so that $1$ is the origin of the path, $N$ its destination, and $i_1, i_2, \dots$ are the nodes travelled. A feasible path is one where all movement constraints and time constraints are fulfilled.


# Parameters and Variables

$f(i,j)$: according to the authors, it is the earliest feasible arrival time at node $i$, provided one can depart along arc $(i,j)$. For now, a better interpretation seems to be that $f(i,j)$ is the arrival time at node $i$ under the current solution for a given path. Written as `f[i][j]`

# Constraints
Arc $a=(i,j)$ is closed for all other trains when a train travels along it from station $i$ to statoin $j$, departing at time $t$ and traveling for duration $d(a)$ . This closing period is denoted as

$$[t, t+d(a)]$$

A station is closed for a security duration $\Delta$ after a train leaves a 
station.

A station has separate capacities for parking and passing through. Only one train can be parked at a station at each moment, but on other train can pass through the station while one other train is parked there.

A route along arc $(i,j)$ is blocked if, for this specific route, the train can not depart from mode $i$ to node $j$. This can have different reasons, which will be explained below.

# Splitting the problem



## Calculating arrival times

The problem can be solved by finding the earliest arrival time at each node, beginning from the start node. Hence, the main task is to calculate an arrival time for a current solution, then updating it if later restrictions make that necessary.

Once the arrival time $f(i,j)$ is known, there are three different possibilities for the arrival time $f(j,k)$:

1. The path is not blocked along arc $(j,k)$. That means that the earliest arrival time $f(j,k)$ is the arrival time $f(i,j)$, plus waiting time at node $i$ and travel time on arc $(i,j)$, for the node $i$ that minimises this sum.
\begin{align*}
f(j,k) = \underset{i,p_i}{\min}[f(i,j) + p_i + t_{ij}]
\end{align*}
We have to account for the possibility that an arc may be reached via different routes.

2. The path is blocked along arc $(j,k)$ and can not be unblocked. In that case, the route can not contain this arc. A route is irreversibly blocked if the train arrives after the closing of the departure time window.

3. A path is blocked along arc $(i,j)$, but can be unblocked. In this case, the train might arrive within either the parking time window of departure time window by extending its parking time at other nodes or taking another route. The last option is not considered by the authors. The first option means that we have now two possible values for $f(i_1, i_2)$: to the already existing solution derived from earlier steps, we add the new one with increased parking times in some node preceding $i_1$.

If an arc is blocked for a current path solution, we write $f(i,j) = \infty$.

## Blocked routes

A path is blocked on arc $(i,j)$ if 

1. Departure is allowed before parking (it is not necessary to wait for departure while parking), but arrival occurs before departure is allowed: $f(i,j) < \gamma_{ij} < \alpha_i$
2. Departure is allowed at some point in time during the parking period (it is possible to wait for departure while parking), but arrival occurs before parking is allowed: $f(i,j) < \alpha_i < \gamma_{ij} < \beta_i$
3. Departure is allowed after the parking period (it is not possible to wait for departure while parking), and arrival occurs before parking: $f(i,j) < \alpha_i < \beta_i < \gamma_{ij}$
4. Arrival occurs after departure is allowed: $\delta_{ij} < f(i,j)$

# Reading in the data

In [142]:
import math
import numpy as np

# all fixed values:
alphaList = [0, 18, 35, 64, 35, 64, 0]
betaList  = [math.inf, 30, 45, 75, 50, 75, math.inf]
gammaList = [10, 10, 15, 28, 26, 32, 35, 43, 70, 30, 48, 72]
deltaList = [15, 25, 25, 35, 34, 60, 50, 50, 99, 80, 55, 80]
nodes     = [1, 2, 3, 4, 5, 6, 7]
arcs      = {1:[2, 3, 5], 2:[4, 6, 7], 3:[2, 4], 4:[7], 5:[2, 6], 6:[7]}

# to facilitate indexing, we zip the lists into dictionaries:
alpha = dict(zip(nodes, alphaList))
beta  = dict(zip(nodes, betaList ))
# gamma and delta have tuples as keys
arccount = 0
gamma = {}
delta = {}
for start in arcs.keys():
    for end in arcs[start]:
        gamma[(start, end)] = gammaList[arccount]
        delta[(start, end)] = deltaList[arccount]
        arccount += 1

# Finding a solution

In [91]:
# Setting feasible arrival times at source to 0:
feasible[1] = {item:0 for (item, value) in list(feasible[1].items())}

# Writing a function to determine whether a route is blocked for arc (i,j):
def isBlocked(alpha, beta, gamma, delta, arrival):
    # check: is a feasible arrival time for i and j already defined?
    if math.isnan(feasible[i][j]):
        raise ValueError('There is no arrival time for the start node yet.')
    # first possibility: current arrival time is higher than delta
    elif feasible[i][j] > depart[i][j][1]:
        return -1 # need three outputs, because this can not be unblocked
    # second possibility: arrival occurs before parking, before departure time
    elif feasible[i][j] < depart[i][j][0] < parking[i][0]:
        return 0 # this can be unblocked
    # third: arrival occurs before departure, before parking time
    elif feasible[i][j] < parking[i][0] < depart[i][j][0]:
        return 0 # this can be unblocked
    else:
        return False

In [ ]:
# Workflow outline:
# - initiate the set of unprocessed paths U to only the source, and the set of useful
#   paths P to an empty set
# - select a path q from the set of unprocessed paths and remove it from U
# - for all nodes w in the network that are connected to the ending of q:
#   - check if w is a feasible extension to q; if it is, add path (q,w) to U
# - add q to P
# - for all v that are an end node to paths in both U and P:
#   - apply dominance criterium
# do all this as long as you can extend the paths in U

# necessary sets:
# - U must be a set of paths. For each path q in U, it must contain information
#   about  the parking and arrival times at each node in q.
# - P must also be a set of paths, which holds the same information as U.

In [165]:
U = [{"route": [1],"parking":0, "arrival":0}]
q = U[0]
q["route"][-1]
arcs[q["route"][-1]]

[2, 3, 5]

In [168]:
q = U[0]
for nextNode in arcs[q["route"][-1]]:
      # getting possible extensions to the last node in the current path
    print(nextNode)

2
3
5
